In [ ]:
# pip install faiss-gpu-cu12

In [ ]:
# pip install langchain-text-splitters

In [ ]:
# pip install langchain-huggingface sentence-transformers

In [ ]:
# pip install langchain-community

In [ ]:
# pip install pypdf

In [ ]:
# pip install jq

In [1]:
import os
import json
import pandas as pd
import random

In [2]:
from typing import List, Optional, Dict

import faiss
import numpy as np
from tqdm import tqdm
from langchain_community.document_loaders import (
    PyPDFLoader,
    TextLoader,
    JSONLoader
)
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

from transformers import AutoTokenizer, AutoModel, AutoModelForCausalLM

import torch
from torch import Tensor
import torch.nn.functional as F

/home/ubuntu/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# pip install matplotlib

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# main_path = '/content/drive/MyDrive/PRACTICUM_DLE/sprint_7/'
main_path = ''

# 1 EDA

In [12]:
data_path = main_path + 'nlp_s3_project/'

data_json_path = data_path + 'arxiv-metadata-s.json'
data_csv_path = data_path + 'test_sample.csv'

data_sample_path = data_path + 'sample_data.json'
data_sample2_path = data_path + 'sample_data2.json'
data_sample3_path = data_path + 'sample_data3.json'
data_sample4_path = data_path + 'sample_data4.json'

In [13]:
# with open(data_path + 'sample_data.json', 'w') as file:
#   json.dump(train_data[:10000], file)

In [14]:
with open(data_json_path, 'r') as file:
  train_data = json.load(file)

In [15]:
with open(data_sample_path, 'r') as file:
  sample_data = json.load(file)

In [16]:
with open(data_sample2_path, 'r') as file:
  sample_data2 = json.load(file)

In [17]:
with open(data_sample3_path, 'r') as file:
  sample_data3 = json.load(file)

In [18]:
with open(data_sample4_path, 'r') as file:
  sample_data4 = json.load(file)

In [19]:
len(sample_data), len(sample_data2), len(sample_data3), len(sample_data4), 

(10000, 505, 100, 100)

In [20]:
df_test = pd.read_csv(data_csv_path)

In [21]:
df_test.shape

(1000, 3)

In [ ]:
# print(df_test[:100]['id'].tolist())

In [ ]:
# our_ids = []
# for i, elem in tqdm(enumerate(train_data)):
#     if elem['id'] in df_test[:100]['id'].tolist():
#         # print(i)
#         our_ids.append(i)

In [ ]:
# our_ids = []
# for j, row in tqdm(df_test[:100].iterrows()):
#     for i, elem in (enumerate(train_data)):
#         if elem['id'] == row['id']:
#             our_ids.append(i)

In [ ]:
# sample_data4 = [train_data[i] for i in our_ids] #+ random.sample(train_data, 100)
# len(sample_data4)

In [ ]:
# sample_data4[0]

In [ ]:
# with open(data_path + 'sample_data4.json', 'w') as file:
#   json.dump(sample_data4, file)

In [22]:
df_test.head()

,id,abstract,query
0,2412.16732,A new platinate was recently discovered when...,What unique composition and decomposition beha...
1,nucl-th/9602019,The production cross sections of various fra...,How does the inclusion of statistical decay af...
2,2501.05500,This survey provides a comprehensive examina...,What are the core components of modern zero-kn...
3,2506.20892,A critical challenge for operating fusion burn...,How does impurity seeding affect the timing an...
4,2208.02031,"In this work, we present the first corpus fo...",What is the primary challenge of the newly dev...


## 1.1 Train data

In [ ]:
df_train = pd.DataFrame(random.sample(train_data, 10000))

In [ ]:
df_train.head(3)

## title

In [ ]:
df_train['title'].apply(lambda x: len(x.split())).hist()

In [ ]:
df_train['title'].str.len().hist()

In [ ]:
df_train['abstract'].apply(lambda x: len(x.split())).hist()

In [ ]:
df_train['abstract'].str.len().hist()

In [ ]:
embedder_name = "Qwen/Qwen3-Embedding-0.6B"

device = "cuda" if torch.cuda.is_available() else "cpu"
emb_tokenizer = AutoTokenizer.from_pretrained(embedder_name)

In [ ]:
rezu = emb_tokenizer(df_train['abstract'].tolist())

In [ ]:
rezu.keys()

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
plt.hist([len(tk) for tk in rezu['input_ids']])

In [ ]:
[len(tk) for tk in rezu['input_ids']]

## submitter

In [ ]:
df_train['submitter'].value_counts()

## tags

In [ ]:
categories_dict = {}
for cats in df_train['categories']:
  for cat in cats.split():
    if cat in categories_dict:
      categories_dict[cat] += 1
    else:
      categories_dict[cat] = 1

In [ ]:
df_cats = pd.DataFrame([[k, v] for k, v in categories_dict.items()], columns=['category', 'count']).sort_values('count', ascending=False).reset_index(drop=True)
df_cats

In [ ]:
df_train['categories'].apply(lambda x: len(x.split())).value_counts().reset_index()

## update_date

In [ ]:
pd.to_datetime(df_train['update_date']).hist()

## versions

In [ ]:
df_train['versions'].apply(lambda x: len(x)).value_counts()

## authors_parsed

In [ ]:
df_train.head(2)

In [ ]:
df_train['authors_parsed'].apply(lambda x: len(x)).value_counts().reset_index()

## 1.2 Test data

In [ ]:
df_test.head()

In [ ]:
df_test.loc[0, 'query']

In [ ]:
df_test.loc[0, 'abstract']

In [ ]:
df_test['abstract'].apply(lambda x: len(x.split())).hist()

In [ ]:
df_test['query'].apply(lambda x: len(x.split())).hist()

# Выводы по EDA

1. как вы будете решать задачу;
- Сделаем RAG+LLM
2. какие подходы к поиску примените и почему;
- Графовая БД index какой-то
3. какие модели выберете и от каких ограничений будете отталкиваться;
- Как в примерах
4. из чего будет состоять ваша система.
- Загрузчик данных
- Ембеддер данных
- БДшка векторная и индекс
- функция поиска релевантных данных к запросу
- функция ранжирования релеввантных
- Формирование комби запроса с учетом релевантных
- Инференс с ЛЛМ
- расчет метрики

# 2 Этап 2. Реализация retrieval-системы

In [ ]:
# from langchain_core.documents import Document
# def load_from_list_of_dicts(data: list[dict]) -> list[Document]:
#     """
#     Converts a list of dictionaries into a list of LangChain Document objects.

#     Each dictionary is expected to have 'page_content' and optional 'metadata' keys.
#     """
#     documents = []
#     for item in data:
#         page_content = item.get('title') + '\n' + item.get('abstract')
#         metadata = {'source': item.get('categories')}
#         # Create a Document object
#         doc = Document(page_content=page_content, metadata=metadata)
#         documents.append(doc)
#     return documents

In [23]:
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["categories"] = record["categories"]
    metadata["id"] = record["id"]
    return metadata

In [24]:
class RAG:

    def __init__(
        self,
        # embedder_name: str = "intfloat/e5-base-v2",
        # reranker_name: str = "cross-encoder/ms-marco-MiniLM-L-12-v2",

        embedder_name: str = "Qwen/Qwen3-Embedding-0.6B",
        reranker_name: str = "Qwen/Qwen3-Reranker-0.6B",

        # chunk_size: int = 500,
        # chunk_overlap: int = 125,
        device: Optional[str] = None,
    ):
        self.device = device or ("cuda"
                                 if torch.cuda.is_available() else "cpu")
        self.emb_tokenizer = AutoTokenizer.from_pretrained(embedder_name)
        self.embedder = AutoModel.from_pretrained(embedder_name).to(
            self.device)
        self.embedder.eval()

        self.rr_tokenizer = AutoTokenizer.from_pretrained(
            reranker_name,
            padding_side='left')
        self.reranker = AutoModelForCausalLM.from_pretrained(
            reranker_name).to(self.device)
        self.reranker.eval()

        # self.text_splitter = RecursiveCharacterTextSplitter(
        #     chunk_size=chunk_size,
        #     chunk_overlap=chunk_overlap,
        # )
        self.index = None
        self.doc_store = []

        # self.max_length = 8192
        self.max_length = 512
        self.token_false_id = self.rr_tokenizer.convert_tokens_to_ids("no")
        self.token_true_id = self.rr_tokenizer.convert_tokens_to_ids("yes")
        prefix = "<|im_start|>system\nJudge whether the Document meets the requirements based on the Query and the Instruct provided. Note that the answer can only be \"yes\" or \"no\".<|im_end|>\n<|im_start|>user\n"
        suffix = "<|im_end|>\n<|im_start|>assistant\n<think>\n\n</think>\n\n"
        self.prefix_tokens = self.rr_tokenizer.encode(prefix,
                                                      add_special_tokens=False)
        self.suffix_tokens = self.rr_tokenizer.encode(suffix,
                                                      add_special_tokens=False)

    def _generate_embeddings(self, texts: List[str]) -> np.ndarray:
        inputs = self.emb_tokenizer(
            texts,
            padding=True,
            truncation=True,
            return_tensors="pt",
            max_length=self.max_length,
        ).to(self.device)

        with torch.no_grad():
            outputs = self.embedder(**inputs)

        inputs.to("cpu")
        embeddings = self.last_token_pool(outputs.last_hidden_state,
                                          inputs.attention_mask).cpu()
        return F.normalize(embeddings, p=2, dim=1).numpy()

    @staticmethod
    def last_token_pool(last_hidden_states: Tensor,
                        attention_mask: Tensor) -> Tensor:
        left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
        if left_padding:
            return last_hidden_states[:, -1]
        else:
            sequence_lengths = attention_mask.sum(dim=1) - 1
            batch_size = last_hidden_states.shape[0]
            return last_hidden_states[
                torch.arange(batch_size, device=last_hidden_states.device),
                sequence_lengths]

    # Define the metadata extraction function.


    def load_and_process_file(self, file_path: str) -> List[Document]:
        """Загрузка и экстракция данных из файлов"""
        ext = os.path.splitext(file_path)[1]
        if ext == ".json":
            loader = JSONLoader(
              file_path=file_path,
              jq_schema='.[]',
              content_key="abstract",
              text_content=True,
              # is_content_key_jq_parsable=True,
              metadata_func=metadata_func
            )
        else:
            raise ValueError(f"Unsupported file type: {ext}")

        docs = loader.load()

        # return self.text_splitter.split_documents(docs)
        return docs


    def build_index(self, file_paths: List[str], batch_size: int = 4) -> None:
        """Строим индекс FAISS"""
        all_docs = []
        for path in file_paths:
            all_docs.extend(self.load_and_process_file(path))
        self.doc_store = all_docs

        # Вычислим numpy-эмбеддинги по батчам
        embeddings = []
        for i in tqdm(range(0, len(all_docs), batch_size)):
            batch = [doc.page_content for doc in all_docs[i:i + batch_size]]
            embeddings.append(self._generate_embeddings(batch))

        embeddings = np.concatenate(embeddings)

        # Инициализируем индекс
        self.index = faiss.IndexFlatL2(embeddings.shape[1])
        self.index.add(embeddings)

    @staticmethod
    def get_detailed_instruct(task_description: str, query: str):
        return f'Instruct: {task_description}\nQuery:{query}'

    @staticmethod
    def format_reranker_instruction(query, doc, instruction=None):
        if instruction is None:
            instruction = 'Given a web search query, retrieve relevant passages that answer the query'
        output = "<Instruct>: {instruction}\n<Query>: {query}\n<Document>: {doc}".format(
            instruction=instruction, query=query, doc=doc)
        return output

    def process_inputs(self, pairs):
        """Обработка данных для реранкера"""
        inputs = self.rr_tokenizer(pairs,
                                   padding=False,
                                   truncation='longest_first',
                                   return_attention_mask=False,
                                   max_length=self.max_length -
                                   len(self.prefix_tokens) -
                                   len(self.suffix_tokens))
        for i, ele in enumerate(inputs['input_ids']):
            inputs['input_ids'][
                i] = self.prefix_tokens + ele + self.suffix_tokens
        inputs = self.rr_tokenizer.pad(inputs,
                                       padding=True,
                                       return_tensors="pt",
                                       max_length=self.max_length)

        # переносим тензоры на девайс ранжирующей модели
        for key in inputs:
            inputs[key] = inputs[key].to(self.device)
        return inputs

    def search(self,
               query: str,
               k: int = 5,
               task: str = None):
        if self.index is None:
            raise ValueError("Index not initialized")

        if task is None:
            task = 'Given a web search query, retrieve relevant passages that answer the query'

        query_embedding = self._generate_embeddings([query])
        distances, indices = self.index.search(query_embedding, k)
        return distances, indices

    @torch.no_grad()
    def compute_logits(self, inputs):
        batch_scores = self.reranker(**inputs).logits[:, -1, :]
        true_vector = batch_scores[:, self.token_true_id]
        false_vector = batch_scores[:, self.token_false_id]
        batch_scores = torch.stack([false_vector, true_vector], dim=1)
        batch_scores = torch.nn.functional.log_softmax(batch_scores, dim=1)
        scores = batch_scores[:, 1].exp().tolist()
        return scores

    def rerank(self, query: str, documents: List[str], batch_size=1):
        pairs = []
        for d in documents:
            pairs.append(self.format_reranker_instruction(query, d))

        scores = []
        for i in range(0, len(pairs), batch_size):

            inputs = self.process_inputs(pairs[i:i + batch_size])
            sc = self.compute_logits(inputs)
            scores.extend(sc)
        return scores

In [25]:
def find_index_next(my_list, target_value, not_found_value):
    return next((i + 1 for i, x in enumerate(my_list) if x == target_value), not_found_value)

In [26]:
# # освободим место на GPU
# torch.cuda.empty_cache()
# rag.embedder.to("cpu")
# rag.reranker.to("cpu")

In [27]:
rag = RAG(device="cuda")

In [28]:
rag.build_index(
    [
        # data_json_path,

        # data_sample_path,
        # data_sample2_path,

        data_sample4_path,
        
    ],
    batch_size=1
)

100%|██████████| 100/100 [00:07<00:00, 13.47it/s]


In [29]:
test_check_idx = 4

In [30]:
q = df_test.loc[test_check_idx, 'query']
answer = df_test.loc[test_check_idx, 'abstract']
id_true = df_test.loc[test_check_idx, 'id']
q, id_true, answer

('What is the primary challenge of the newly developed German ADR detection corpus in terms of its annotation?',
 '2208.02031',
 '  In this work, we present the first corpus for German Adverse Drug Reaction\n(ADR) detection in patient-generated content. The data consists of 4,169 binary\nannotated documents from a German patient forum, where users talk about health\nissues and get advice from medical doctors. As is common in social media data\nin this domain, the class labels of the corpus are very imbalanced. This and a\nhigh topic imbalance make it a very challenging dataset, since often, the same\nsymptom can have several causes and is not always related to a medication\nintake. We aim to encourage further multi-lingual efforts in the domain of ADR\ndetection and provide preliminary experiments for binary classification using\ndifferent methods of zero- and few-shot learning based on a multi-lingual\nmodel. When fine-tuning XLM-RoBERTa first on English patient forum data and\nthen o

In [32]:
for i, elem in enumerate(sample_data4):
    if elem['id'] == id_true:
        print(i, elem['id'])
        break

4 2208.02031


In [33]:
k_cands = 5

In [34]:
_, I = rag.search(q, k=k_cands)

In [35]:
candidates = [rag.doc_store[i].page_content for i in I[0]]

In [36]:
I[0]

array([ 4, 86, 18, 20, 80])

In [37]:
rag.doc_store[39]

Document(metadata={'source': '/home/ubuntu/practicum_dle_sprint_7/nlp_s3_project/sample_data4.json', 'seq_num': 40, 'categories': 'astro-ph', 'id': '0809.4362'}, page_content='  We used the red clump stars from the photometric data of the Optical\nGravitational Lensing Experiment(OGLE II) survey and the Magellanic Cloud\nPhotometric Survey (MCPS) for both the Clouds to estimate the depth.The\nobserved dispersion in the magnitude and colour distribution of red clump stars\nis used to estimate the depth, after correcting for population effects,\ninternal reddening within the Clouds and photometric errors.The observed\ndispersion due to the line of sight depth ranges from 0.023 mag to 0.45 mag (a\ndepth of 500 pc to 10.4 kpc) for LMC and from 0.025 to 0.34 magnitude(a depth\nof 670 pc to 9.53 kpc).The depth profile of the LMC bar indicates that it is\nflared.The average depth in the bar region is 4.0$\\pm$1.4 kpc. The northern\ndisk is found to have depth(4.17$\\pm$0.97 kpc)larger than th

In [38]:
relevant_flags = [rag.doc_store[i].metadata['id'] == id_true for i in I[0]]
find_index_next(relevant_flags, True, np.inf)

1

In [39]:
k_ranked = 100

In [40]:
candidates[0]

'  In this work, we present the first corpus for German Adverse Drug Reaction\n(ADR) detection in patient-generated content. The data consists of 4,169 binary\nannotated documents from a German patient forum, where users talk about health\nissues and get advice from medical doctors. As is common in social media data\nin this domain, the class labels of the corpus are very imbalanced. This and a\nhigh topic imbalance make it a very challenging dataset, since often, the same\nsymptom can have several causes and is not always related to a medication\nintake. We aim to encourage further multi-lingual efforts in the domain of ADR\ndetection and provide preliminary experiments for binary classification using\ndifferent methods of zero- and few-shot learning based on a multi-lingual\nmodel. When fine-tuning XLM-RoBERTa first on English patient forum data and\nthen on the new German data, we achieve an F1-score of 37.52 for the positive\nclass. We make the dataset and models publicly available

In [41]:
scores = rag.rerank(q, candidates)
array = np.array(scores)
indices = np.argsort(array)[::-1][:k_ranked]

You're using a Qwen2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/ubuntu/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2714: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(


In [42]:
indices

array([0, 1, 2, 4, 3])

In [44]:
ranked_ids = [int(I[0][i]) for i in indices]
ranked_ids

[4, 86, 18, 80, 20]

In [45]:
relevant_flags = [rag.doc_store[i].metadata['id'] == id_true for i in ranked_ids]

In [46]:
find_index_next(relevant_flags, True, np.inf)

1

In [47]:
K_mrr = 5
N_cands_rerank = 5

mrr5_list = []
for j, row in tqdm(df_test[:10].iterrows()):
  # print(index, row['id'])
  q = row['query']
  id_true = row['id']


  _, I = rag.search(q, k=N_cands_rerank)
  candidates = [rag.doc_store[i].page_content for i in I[0]]
  scores = rag.rerank(q, candidates)
  array = np.array(scores)
  indices = np.argsort(array)[::-1][:K_mrr]
  ranked_ids = [int(I[0][i]) for i in indices]

  relevant_flags = [rag.doc_store[i].metadata['id'] == id_true for i in ranked_ids]   # i in I[0]]
  relev_index = find_index_next(relevant_flags, True, np.inf)

  mrr5_step = 1 / relev_index
  if mrr5_step > 0 :
    print(f'j={j}, relev_index = {relev_index}, mrr5_step={mrr5_step}')
  mrr5_list.append(mrr5_step)

0it [00:00, ?it/s]/home/ubuntu/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:2714: UserWarning: `max_length` is ignored when `padding`=`True` and there is no truncation strategy. To pad to max length, use `padding='max_length'`.
  warnings.warn(
1it [00:00,  1.34it/s]

j=0, relev_index = 1, mrr5_step=1.0


2it [00:01,  1.55it/s]

j=1, relev_index = 1, mrr5_step=1.0


3it [00:01,  1.66it/s]

...

j=7, relev_index = 1, mrr5_step=1.0


9it [00:05,  1.67it/s]

j=8, relev_index = 1, mrr5_step=1.0


10it [00:05,  1.71it/s]

j=9, relev_index = 1, mrr5_step=1.0


In [ ]:
MRR5 = sum(mrr5_list) / len(mrr5_list)
MRR5

In [ ]:
1 / np.inf

In [ ]:
I

In [ ]:
relevant_flags = [rag.doc_store[i].metadata['id'] == id_true for i in I[0]]
relevant_flags

In [ ]:
relevant_flags = [True, False, False, True, False]

In [ ]:
find_index_next(relevant_flags, True, 5)

In [ ]:
find_index_or_none_next(relevant_flags, True)

In [ ]:
relevant_flags.index(1)

In [ ]:
mrr@5 =

In [ ]:
next((item for item in relevant_flags if item), False)

In [ ]:
[rag.doc_store[i].metadata['id'] for i in I[0]]

In [ ]:
candidates = [rag.doc_store[i].page_content for i in I[0]]

In [ ]:
rag.doc_store[23].metadata['id']